In [1]:
import bql
import bqplot as bqp
import ipydatagrid as ipd
import ipywidgets as widgets
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from datetime import datetime, timedelta
from IPython.display import display, clear_output
from plotly.subplots import make_subplots

In [2]:
# Connect to BQL
bq = bql.Service()

In [10]:
# Define the file path
file_path = 'altd_coverage_list_20250319.xlsx'

# Read the 'coverage_list' sheet, skipping the first 5 rows
coverage_list = pd.read_excel(file_path, sheet_name='ALTD Coverage', skiprows=5).dropna()

# Read the 'brand_mapping' sheet, skipping the first 5 rows
brand_mapping = pd.read_excel(file_path, sheet_name='ALTD Coverage Brands', skiprows=5).dropna()

In [14]:
public_coverage = coverage_list[coverage_list['Ownership'] == 'Public']
public_coverage.head()

,Ticker,Company,Ownership,Domicile,Bloomberg Second Measure,Placer.ai,Similarweb
325,139480 KS Equity,E-MART Inc,Public,KR,Y,Y,Y
497,1910 HK Equity,Samsonite International SA,Public,US,Y,Y,N
658,2661Z US Equity,Massachusetts Bay Transportation Authority,Public,US,Y,N,N
663,27588MF US Equity,Utah Transit Authority,Public,US,Y,N,N
699,3382 JP Equity,Seven & i Holdings Co Ltd,Public,JP,Y,Y,N


In [17]:
public_brands = brand_mapping[brand_mapping['Ownership'] == 'Public']
public_brands.head()

,Ticker,Company,Brand,Ownership,Domicile,Bloomberg Second Measure,Placer.ai,Similarweb
519,139480 KS Equity,E-MART Inc,Bristol Farms Brand,Public,KR,Y,N,Y
520,139480 KS Equity,E-MART Inc,EMart24,Public,KR,N,N,Y
521,139480 KS Equity,E-MART Inc,Lazy Acres Natural Market,Public,KR,Y,Y,Y
522,139480 KS Equity,E-MART Inc,Metropolitan Market,Public,KR,Y,N,Y
523,139480 KS Equity,E-MART Inc,New Leaf Community Markets,Public,KR,Y,N,Y


In [90]:
def get_brand_data(universe, brands, brand_breakout=False):
    # Define the date range for the last ~545 days
    date_range = bq.func.range('-545D', '0D')
    
    # Create a dictionary to hold the data items for each brand
    data_items = {}
    if brand_breakout:
        for brand in brands:
            # Escape any single quotes in the brand name
            safe_brand = brand.replace("'", "\\'")
            # Create the observed_sales data item for this brand
            data_items[brand] = bq.data.observed_sales(
                brand=safe_brand,
                dates=date_range
            )
    else:
        brands=['Online','In_Store']
        for brand in brands:
            # Create the observed_sales data item for this brand
            data_items[brand] = bq.data.observed_sales(
                channel=brand,
                dates=date_range
            )
    
    # Create and execute the request for the specified universe and data items
    request = bql.Request(universe, data_items)
    response = bq.execute(request)
    
    # Concatenate the resulting DataFrames side by side
    df = pd.concat([item.df() for item in response], axis=1)
    
    # Remove any duplicated columns
    df = df.loc[:, ~df.columns.duplicated()]
    
    # Drop unnecessary columns (like SOURCE, BRAND, DATE, PER, and CURRENCY)
    columns_to_drop = [col for col in ['SOURCE', 'BRAND','CHANNEL'] if col in df.columns]
    daily_data = df.drop(columns=columns_to_drop).drop(['DATE', 'PER', 'CURRENCY'], axis=1)
    
    # Retrieve quarter metadata (sales_rev_turn returns a dict-like series for each)
    current_q_start = bq.data.sales_rev_turn(
        fa_period_type='Q', 
        fa_period_offset='0'
    )['PERIOD_END_DATE']
    
    # Unreported quarter end date
    current_q_end = bq.data.sales_rev_turn(
        fa_period_type='Q', 
        fa_period_offset='1'
    )['PERIOD_END_DATE']
    
    # Comparison quarter start date
    comparison_q_start = bq.data.sales_rev_turn(
        fa_period_type='Q', 
        fa_period_offset='-4'
    )['PERIOD_END_DATE']
    
    # Comparison quarter end date
    comparison_q_end = bq.data.sales_rev_turn(
        fa_period_type='Q', 
        fa_period_offset='-3'
    )['PERIOD_END_DATE']

    data_items_meta = {
        'Current Quarter Start': current_q_start,
        'Current Quarter End': current_q_end,
        'Comparison Quarter Start': comparison_q_start,
        'Comparison Quarter End': comparison_q_end,
        'Total Days in Current Quarter': current_q_end - current_q_start,
        'Days in Comp Quarter': comparison_q_end - comparison_q_start
    }
    
    with_params = {
        'act_est_mapping': 'precise',
        'filing_status': 'MRC'
    }
    
    request = bql.Request(universe, data_items_meta, with_params=with_params, preferences=None)
    company_data = bq.execute(request)
    
    quarter_meta = pd.concat([data_item.df() for data_item in company_data], axis=1)

    # Extract date boundaries from quarter_meta (assuming one row)
    curr_start = pd.to_datetime(quarter_meta['Current Quarter Start'].iloc[0])
    curr_end   = pd.to_datetime(quarter_meta['Current Quarter End'].iloc[0])
    comp_start = pd.to_datetime(quarter_meta['Comparison Quarter Start'].iloc[0])
    comp_end   = pd.to_datetime(quarter_meta['Comparison Quarter End'].iloc[0])
    
    # Filter the DataFrame into current and comparison quarter data
    curr_quarter = daily_data[(daily_data['PERIOD_END_DATE'] >= curr_start) & 
                              (daily_data['PERIOD_END_DATE'] <= curr_end)].copy()
    comp_quarter = daily_data[(daily_data['PERIOD_END_DATE'] >= comp_start) & 
                              (daily_data['PERIOD_END_DATE'] <= comp_end)].copy()

    quarter_meta['Days Elapsed in Current Quarter'] = len(curr_quarter)
    
    # Identify numeric columns (the sales/brand data)
    brand_cols = curr_quarter.select_dtypes(include=[np.number]).columns.tolist()
    
    # Compute cumulative sums for each brand (QTD values)
    for col in brand_cols:
        curr_quarter[col] = curr_quarter[col].cumsum()
        comp_quarter[col] = comp_quarter[col].cumsum()
    
    # Reset the index so we can merge on the row number
    curr_quarter = curr_quarter.reset_index(drop=True)
    comp_quarter = comp_quarter.reset_index(drop=True)
    
    # Merge the current and comparison quarters on the index (row number)
    merged = pd.merge(
        curr_quarter, comp_quarter,
        left_index=True, right_index=True,
        suffixes=('_curr', '_comp')
    )
    
    # Compute the QTD year-over-year growth for each brand.
    # For example: Walmart_YoY_QTD = (Walmart_curr / Walmart_comp) - 1
    for brand in brands:
        curr_col = brand + '_curr'
        comp_col = brand + '_comp'
        merged[brand + '_YoY_QTD'] = merged[curr_col] / merged[comp_col] - 1
    
    # Rename the current quarter date column for clarity
    merged.rename(columns={'PERIOD_END_DATE_curr': 'PERIOD_END_DATE'}, inplace=True)
    
    # Set the index to be PERIOD_END_DATE
    merged.set_index('PERIOD_END_DATE', inplace=True)
    
    # Drop all columns that do not end in 'YoY_QTD'
    merged_growth = merged[[col for col in merged.columns if col.endswith('YoY_QTD')]].copy()
    
    # Rename columns to drop the '_YoY_QTD' suffix so they are just the brand names
    merged_growth.rename(columns=lambda x: x.replace('_YoY_QTD', ''), inplace=True)

    share = pd.DataFrame()
    #share of sales
    share['Total'] = curr_quarter.set_index('PERIOD_END_DATE').sum(axis=1)

    for brand in brands:
        share[brand] = curr_quarter.set_index('PERIOD_END_DATE')[brand] / share['Total']
    share.drop(columns=['Total'],inplace=True)
    
    return merged_growth, share, quarter_meta

merged_growth, share, quarter_meta = get_brand_data(
    universe='WMT US Equity', 
    brands=["Walmart", "Sam's Club", "Walmart Gas", "Shoes.com"],
    #brand_breakout=True
)
share.head()

,Online,In_Store
PERIOD_END_DATE,,
2025-01-31,0.256793,0.743207
2025-02-01,0.244321,0.755679
2025-02-02,0.247036,0.752964
2025-02-03,0.245138,0.754862
2025-02-04,0.244543,0.755457


In [91]:
quarter_meta

,Current Quarter Start,Current Quarter End,Comparison Quarter Start,Comparison Quarter End,Total Days in Current Quarter,Days in Comp Quarter,Days Elapsed in Current Quarter
ID,,,,,,,
WMT US Equity,2025-01-31,2025-04-30,2024-01-31,2024-04-30,89.0,90.0,42
